<a href="https://colab.research.google.com/github/viincci/Flask-Task-manager-project/blob/main/FTMPColabNotebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
!pip install -q flask-ngrok pyngrok flask flask-sqlalchemy

In [14]:
import os
from google.colab import drive
from pyngrok import ngrok

In [15]:
# You can set your ngrok auth token here or store it in Drive
ngrok.set_auth_token("YOUR_NGROK_AUTH_TOKEN")

# Optionally mount Google Drive if you want to store your app files there
drive.mount('/content/drive')

Mounted at /content/drive


In [16]:
# Import required libraries for our Flask app
from flask import Flask, render_template, request, redirect, url_for, flash, session
from flask_sqlalchemy import SQLAlchemy
import datetime
import re
from werkzeug.security import generate_password_hash, check_password_hash

In [17]:
app = Flask(__name__)
app.config['SECRET_KEY'] = 'colab_secret_key'
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///tasks.db'
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False

In [18]:
db = SQLAlchemy(app)

# Define your database models
class User(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(80), unique=True, nullable=False)
    password_hash = db.Column(db.String(256), nullable=False)
    tasks = db.relationship('Task', backref='owner', lazy=True)

    def set_password(self, password):
        self.password_hash = generate_password_hash(password)

    def check_password(self, password):
        return check_password_hash(self.password_hash, password)

class Task(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    title = db.Column(db.String(100), nullable=False)
    description = db.Column(db.Text)
    completed = db.Column(db.Boolean, default=False)
    created_at = db.Column(db.DateTime, default=datetime.datetime.utcnow)
    user_id = db.Column(db.Integer, db.ForeignKey('user.id'), nullable=False)

In [19]:
# Create a directory for templates
import os
if not os.path.exists('templates'):
    os.makedirs('templates')

# Create basic templates
with open('templates/layout.html', 'w') as f:
    f.write('''
<!DOCTYPE html>
<html>
<head>
    <title>Flask Task Manager</title>
    <meta name="viewport" content="width=device-width, initial-scale=1">
    <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0-alpha1/dist/css/bootstrap.min.css" rel="stylesheet">
</head>
<body>
    <nav class="navbar navbar-expand-lg navbar-dark bg-dark">
        <div class="container">
            <a class="navbar-brand" href="/">Task Manager</a>
            <button class="navbar-toggler" type="button" data-bs-toggle="collapse" data-bs-target="#navbarNav">
                <span class="navbar-toggler-icon"></span>
            </button>
            <div class="collapse navbar-collapse" id="navbarNav">
                <ul class="navbar-nav ms-auto">
                    {% if 'user_id' in session %}
                    <li class="nav-item">
                        <a class="nav-link" href="{{ url_for('add_task') }}">Add Task</a>
                    </li>
                    <li class="nav-item">
                        <a class="nav-link" href="{{ url_for('logout') }}">Logout</a>
                    </li>
                    {% else %}
                    <li class="nav-item">
                        <a class="nav-link" href="{{ url_for('login') }}">Login</a>
                    </li>
                    <li class="nav-item">
                        <a class="nav-link" href="{{ url_for('register') }}">Register</a>
                    </li>
                    {% endif %}
                </ul>
            </div>
        </div>
    </nav>

    <div class="container mt-4">
        {% with messages = get_flashed_messages(with_categories=true) %}
        {% if messages %}
            {% for category, message in messages %}
            <div class="alert alert-{{ category }}">
                {{ message }}
            </div>
            {% endfor %}
        {% endif %}
        {% endwith %}

        {% block content %}{% endblock %}
    </div>

    <script src="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0-alpha1/dist/js/bootstrap.bundle.min.js"></script>
</body>
</html>
    ''')
with open('templates/login.html', 'w') as f:
    f.write('''
{% extends "layout.html" %}
{% block content %}
<div class="row justify-content-center">
    <div class="col-md-6">
        <div class="card">
            <div class="card-header">Login</div>
            <div class="card-body">
                <form method="POST">
                    <div class="mb-3">
                        <label for="username" class="form-label">Username</label>
                        <input type="text" class="form-control" id="username" name="username" required>
                    </div>
                    <div class="mb-3">
                        <label for="password" class="form-label">Password</label>
                        <input type="password" class="form-control" id="password" name="password" required>
                    </div>
                    <button type="submit" class="btn btn-primary">Login</button>
                </form>
            </div>
        </div>
    </div>
</div>
{% endblock %}
    ''')

with open('templates/register.html', 'w') as f:
    f.write('''
{% extends "layout.html" %}
{% block content %}
<div class="row justify-content-center">
    <div class="col-md-6">
        <div class="card">
            <div class="card-header">Register</div>
            <div class="card-body">
                <form method="POST">
                    <div class="mb-3">
                        <label for="username" class="form-label">Username</label>
                        <input type="text" class="form-control" id="username" name="username" required>
                    </div>
                    <div class="mb-3">
                        <label for="password" class="form-label">Password</label>
                        <input type="password" class="form-control" id="password" name="password" required>
                        <div class="form-text">Must be at least 8 characters with uppercase, number, and special symbol.</div>
                    </div>
                    <button type="submit" class="btn btn-primary">Register</button>
                </form>
            </div>
        </div>
    </div>
</div>
{% endblock %}
    ''')
with open('templates/task_list.html', 'w') as f:
    f.write('''
{% extends "layout.html" %}
{% block content %}
<h1>Your Tasks</h1>
<div class="mb-3">
    <a href="{{ url_for('add_task') }}" class="btn btn-primary">Add New Task</a>
</div>

{% if tasks %}
<div class="row">
    {% for task in tasks %}
    <div class="col-md-4 mb-3">
        <div class="card {% if task.completed %}bg-light{% endif %}">
            <div class="card-body">
                <h5 class="card-title">{{ task.title }}</h5>
                <p class="card-text">{{ task.description }}</p>
                <p class="card-text"><small class="text-muted">Created: {{ task.created_at.strftime('%Y-%m-%d') }}</small></p>
                <div class="d-flex justify-content-between">
                    <a href="{{ url_for('edit_task', task_id=task.id) }}" class="btn btn-sm btn-primary">Edit</a>
                    <a href="{{ url_for('toggle_task', task_id=task.id) }}" class="btn btn-sm {% if task.completed %}btn-warning{% else %}btn-success{% endif %}">
                        {% if task.completed %}Mark Incomplete{% else %}Complete{% endif %}
                    </a>
                    <a href="{{ url_for('delete_task', task_id=task.id) }}" class="btn btn-sm btn-danger" onclick="return confirm('Are you sure?')">Delete</a>
                </div>
            </div>
        </div>
    </div>
    {% endfor %}
</div>
{% else %}
<div class="alert alert-info">You don't have any tasks yet.</div>
{% endif %}
{% endblock %}
    ''')

with open('templates/add_task.html', 'w') as f:
    f.write('''
{% extends "layout.html" %}
{% block content %}
<div class="row justify-content-center">
    <div class="col-md-8">
        <div class="card">
            <div class="card-header">Add New Task</div>
            <div class="card-body">
                <form method="POST">
                    <div class="mb-3">
                        <label for="title" class="form-label">Title</label>
                        <input type="text" class="form-control" id="title" name="title" required>
                    </div>
                    <div class="mb-3">
                        <label for="description" class="form-label">Description</label>
                        <textarea class="form-control" id="description" name="description" rows="3"></textarea>
                    </div>
                    <div class="d-flex justify-content-between">
                        <a href="{{ url_for('home') }}" class="btn btn-secondary">Cancel</a>
                        <button type="submit" class="btn btn-primary">Add Task</button>
                    </div>
                </form>
            </div>
        </div>
    </div>
</div>
{% endblock %}
    ''')

with open('templates/edit_task.html', 'w') as f:
    f.write('''
{% extends "layout.html" %}
{% block content %}
<div class="row justify-content-center">
    <div class="col-md-8">
        <div class="card">
            <div class="card-header">Edit Task</div>
            <div class="card-body">
                <form method="POST">
                    <div class="mb-3">
                        <label for="title" class="form-label">Title</label>
                        <input type="text" class="form-control" id="title" name="title" value="{{ task.title }}" required>
                    </div>
                    <div class="mb-3">
                        <label for="description" class="form-label">Description</label>
                        <textarea class="form-control" id="description" name="description" rows="3">{{ task.description }}</textarea>
                    </div>
                    <div class="d-flex justify-content-between">
                        <a href="{{ url_for('home') }}" class="btn btn-secondary">Cancel</a>
                        <button type="submit" class="btn btn-primary">Update Task</button>
                    </div>
                </form>
            </div>
        </div>
    </div>
</div>
{% endblock %}
    ''')

In [20]:

# Define your routes
@app.route('/register', methods=['GET', 'POST'])
def register():
    if 'user_id' in session:
        return redirect(url_for('home'))

    if request.method == 'POST':
        username = request.form['username']
        password = request.form['password']

        # Simple password validation
        if len(password) < 8 or not any(c.isupper() for c in password) or not any(c.isdigit() for c in password) or not any(c in '!@#$%^&*(),.?":{}|<>' for c in password):
            flash("Password must be at least 8 characters long, include one uppercase letter, one number, and one special symbol.", "danger")
            return redirect(url_for('register'))

        # Check if the username already exists
        existing_user = User.query.filter_by(username=username).first()
        if existing_user:
            flash("Username already exists! Please choose another one.", "danger")
            return redirect(url_for('register'))

        # Create new user
        new_user = User(username=username)
        new_user.set_password(password)

        try:
            db.session.add(new_user)
            db.session.commit()
            flash('Registration successful! Please log in.', 'success')
            return redirect(url_for('login'))
        except Exception as e:
            db.session.rollback()
            flash(f'An error occurred: {str(e)}', 'danger')

    return render_template('register.html')

@app.route('/')
def home():
    if 'user_id' not in session:
        return redirect(url_for('login'))

    user_tasks = Task.query.filter_by(user_id=session['user_id']).order_by(Task.created_at.desc()).all()
    return render_template('task_list.html', tasks=user_tasks)

@app.route('/add', methods=['GET', 'POST'])
def add_task():
    if 'user_id' not in session:
        return redirect(url_for('login'))

    if request.method == 'POST':
        title = request.form['title']
        description = request.form['description']

        if not title:
            flash('Title is required!', 'danger')
            return redirect(url_for('add_task'))

        new_task = Task(
            title=title,
            description=description,
            user_id=session['user_id']
        )

        try:
            db.session.add(new_task)
            db.session.commit()
            flash('Task added successfully!', 'success')
            return redirect(url_for('home'))
        except Exception as e:
            db.session.rollback()
            flash(f'An error occurred: {str(e)}', 'danger')

    return render_template('add_task.html')

@app.route('/edit/<int:task_id>', methods=['GET', 'POST'])
def edit_task(task_id):
    if 'user_id' not in session:
        return redirect(url_for('login'))

    task = Task.query.get_or_404(task_id)

    # Security check
    if task.user_id != session['user_id']:
        flash('You do not have permission to edit this task', 'danger')
        return redirect(url_for('home'))

    if request.method == 'POST':
        title = request.form['title']

        if not title:
            flash('Title is required!', 'danger')
            return redirect(url_for('edit_task', task_id=task_id))

        task.title = title
        task.description = request.form['description']

        try:
            db.session.commit()
            flash('Task updated successfully!', 'success')
            return redirect(url_for('home'))
        except Exception as e:
            db.session.rollback()
            flash(f'An error occurred: {str(e)}', 'danger')

    return render_template('edit_task.html', task=task)

@app.route('/toggle/<int:task_id>')
def toggle_task(task_id):
    if 'user_id' not in session:
        return redirect(url_for('login'))

    task = Task.query.get_or_404(task_id)

    # Security check
    if task.user_id != session['user_id']:
        flash('You do not have permission to modify this task', 'danger')
        return redirect(url_for('home'))

    task.completed = not task.completed

    try:
        db.session.commit()
        status = "completed" if task.completed else "marked as incomplete"
        flash(f'Task {status}!', 'success')
    except Exception as e:
        db.session.rollback()
        flash(f'An error occurred: {str(e)}', 'danger')

    return redirect(url_for('home'))

@app.route('/delete/<int:task_id>')
def delete_task(task_id):
    if 'user_id' not in session:
        return redirect(url_for('login'))

    task = Task.query.get_or_404(task_id)

    # Security check
    if task.user_id != session['user_id']:
        flash('You do not have permission to delete this task', 'danger')
        return redirect(url_for('home'))

    try:
        db.session.delete(task)
        db.session.commit()
        flash('Task deleted successfully!', 'success')
    except Exception as e:
        db.session.rollback()
        flash(f'An error occurred: {str(e)}', 'danger')

    return redirect(url_for('home'))

@app.route('/login', methods=['GET', 'POST'])
def login():
    if 'user_id' in session:
        return redirect(url_for('home'))

    if request.method == 'POST':
        username = request.form['username']
        password = request.form['password']

        user = User.query.filter_by(username=username).first()

        if user and user.check_password(password):
            session['user_id'] = user.id
            session['username'] = user.username
            flash('Login successful!', 'success')
            return redirect(url_for('home'))
        else:
            flash('Invalid credentials!', 'danger')

    return render_template('login.html')

@app.route('/logout')
def logout():
    session.clear()
    flash('Logged out successfully!', 'success')
    return redirect(url_for('login'))

In [21]:

# Create database tables
with app.app_context():
    db.create_all()

    # Create a default admin user if none exists
    if not User.query.filter_by(username='admin').first():
        admin = User(username='admin')
        admin.set_password('Admin123!')
        db.session.add(admin)
        db.session.commit()

In [22]:
# Start ngrok tunnel to expose the app to the internet
public_url = ngrok.connect(5000).public_url
print(f" * Running on {public_url}")

# Modify the app configuration to use the public URL
app.config['BASE_URL'] = public_url

 * Running on https://8e7c-34-82-15-211.ngrok-free.app


In [23]:
app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [20/Mar/2025 10:29:00] "GET / HTTP/1.1" 302 -
INFO:werkzeug:127.0.0.1 - - [20/Mar/2025 10:29:00] "GET /login HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Mar/2025 10:29:01] "GET / HTTP/1.1" 302 -
INFO:werkzeug:127.0.0.1 - - [20/Mar/2025 10:29:01] "GET /login HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Mar/2025 10:29:02] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [20/Mar/2025 10:29:27] "POST /login HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Mar/2025 10:29:58] "POST /login HTTP/1.1" 302 -
INFO:werkzeug:127.0.0.1 - - [20/Mar/2025 10:29:59] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Mar/2025 10:30:01] "GET /add HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Mar/2025 10:30:13] "POST /add HTTP/1.1" 302 -
INFO:werkzeug:12